# Finetuning Bert

In [17]:
# Install library that is not preinstalled on Colab.
!pip install datasets

In [18]:
# Import required libraries.
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer
import datasets as ds
from datasets import load_dataset
import numpy as np
import sys

In [19]:
# Print library versions so enviornment can be reproduced elsewhere.
print("python version: ", sys.version)
print("tensorflow version: ", tf.__version__)
print("numpy version: ", np.__version__)
print("datasets version: ", ds.__version__)

python version:  3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
tensorflow version:  2.15.0
numpy version:  1.25.2
datasets version:  2.20.0


In [20]:
# Defining our base BERT model.
model = TFAutoModel.from_pretrained("bert-base-uncased")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [21]:
# Defining our tokeniser.
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [22]:
# Loading in the dataset and observing its features.
dataset = load_dataset("duxprajapati/symptom-disease-dataset")
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 5634
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1409
    })
})

# Adding validation set

In [23]:
# Taking a portion of the training data and using it as validation data.
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict

train_texts = dataset['train']['text']
train_labels = dataset['train']['label']

train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_texts, train_labels, test_size=0.18, random_state=42  # Adjust test_size for your validation size
)

train_dict = {"text": train_texts, "label": train_labels}
train_dataset = Dataset.from_dict(train_dict)

validation_dict = {"text": val_texts, "label": val_labels}
validation_dataset = Dataset.from_dict(validation_dict)

# Combine the datasets into a DatasetDict
dataset = DatasetDict({
    'train': train_dataset,
    'validation': validation_dataset,
    'test': dataset['test']
})

# Print the DatasetDict to verify
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 4619
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1015
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1409
    })
})


# Finetuning continued

In [24]:
# Function to tokenise a batch.
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True, max_length=128)

In [25]:
# Tokenising the whole dataset using the batch tokenisation function.
dataset_encoded = dataset.map(tokenize, batched=True, batch_size=None)
dataset_encoded

Map:   0%|          | 0/4619 [00:00<?, ? examples/s]

Map:   0%|          | 0/1015 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4619
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1015
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1409
    })
})

In [26]:
# setting 'input_ids', 'attention_mask', 'token_type_ids', and 'label'
# to the tensorflow format. Now if you access this dataset you will get these
# columns in `tf.Tensor` format
dataset_encoded.set_format('tf',
                            columns=['input_ids', 'attention_mask', 'token_type_ids', 'label'])
dataset_encoded

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4619
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1015
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1409
    })
})

In [27]:
# setting BATCH_SIZE to 64.
BATCH_SIZE = 64

def order(inp):
    '''
    This function will group all the inputs of BERT
    into a single dictionary and then output it with
    labels.
    '''
    print(f"type of input of order: {type(inp)}")
    data = list(inp.values())
    print(f"data[1]: {data[1]}")
    print(f"data[2]: {data[2]}")
    print(f"data[3]: {data[3]}")
    return {
        'input_ids': data[1],
        'attention_mask': data[3],
        'token_type_ids': data[2]
    }, data[0]

# converting train split of dataset to tensorflow format
train_dataset = tf.data.Dataset.from_tensor_slices(dataset_encoded['train'][:])
train_dataset

<_TensorSliceDataset element_spec={'label': TensorSpec(shape=(), dtype=tf.int64, name=None), 'input_ids': TensorSpec(shape=(128,), dtype=tf.int64, name=None), 'token_type_ids': TensorSpec(shape=(128,), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(128,), dtype=tf.int64, name=None)}>

In [28]:
# set batch_size and shuffle
train_dataset = train_dataset.batch(BATCH_SIZE).shuffle(1000)
train_dataset

<_ShuffleDataset element_spec={'label': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'input_ids': TensorSpec(shape=(None, 128), dtype=tf.int64, name=None), 'token_type_ids': TensorSpec(shape=(None, 128), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, 128), dtype=tf.int64, name=None)}>

In [29]:
# map the `order` function
train_dataset = train_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset

type of input of order: <class 'dict'>
data[1]: Tensor("args_1:0", shape=(None, 128), dtype=int64)
data[2]: Tensor("args_3:0", shape=(None, 128), dtype=int64)
data[3]: Tensor("args_0:0", shape=(None, 128), dtype=int64)


<_ParallelMapDataset element_spec=({'input_ids': TensorSpec(shape=(None, 128), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, 128), dtype=tf.int64, name=None), 'token_type_ids': TensorSpec(shape=(None, 128), dtype=tf.int64, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [30]:
# ... doing the same for test set ...
test_dataset = tf.data.Dataset.from_tensor_slices(dataset_encoded['test'][:])
test_dataset = test_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

type of input of order: <class 'dict'>
data[1]: Tensor("args_1:0", shape=(None, 128), dtype=int64)
data[2]: Tensor("args_3:0", shape=(None, 128), dtype=int64)
data[3]: Tensor("args_0:0", shape=(None, 128), dtype=int64)


In [31]:
# Same for validation set as well.
validation_dataset = tf.data.Dataset.from_tensor_slices(dataset_encoded['validation'][:])
validation_dataset = validation_dataset.batch(BATCH_SIZE)
validation_dataset = validation_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

type of input of order: <class 'dict'>
data[1]: Tensor("args_1:0", shape=(None, 128), dtype=int64)
data[2]: Tensor("args_3:0", shape=(None, 128), dtype=int64)
data[3]: Tensor("args_0:0", shape=(None, 128), dtype=int64)


In [32]:
# Visualising data format.
inp, out = next(iter(train_dataset)) # a batch from train_dataset
print(inp, '\n\n', out)

{'input_ids': <tf.Tensor: shape=(64, 128), dtype=int64, numpy=
array([[  101, 16342,  1010, ...,     0,     0,     0],
       [  101,  1045,  1005, ...,     0,     0,     0],
       [  101, 24780,  1010, ...,     0,     0,     0],
       ...,
       [  101,  2009,  8450, ...,     0,     0,     0],
       [  101,  6740,  1035, ...,     0,     0,     0],
       [  101,  8030,  1997, ...,     0,     0,     0]])>, 'attention_mask': <tf.Tensor: shape=(64, 128), dtype=int64, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>, 'token_type_ids': <tf.Tensor: shape=(64, 128), dtype=int64, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])>} 

 tf.Tensor(
[ 511  234  718  394    

In [33]:
# Defining our final model, that takes the base BERT model, and adds an output dense layer for predictions.
class BERTForClassification(tf.keras.Model):

    def __init__(self, bert_model, num_classes):
        super().__init__()
        self.bert = bert_model
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.bert(inputs)[1]
        return self.fc(x)

In [34]:
# Defining an instance of the model, and initialising callbacks.
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau

'''
Early stopping allows us to stop training when the validation loss stops
improving. This reduces overfitting.
'''
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=4,
                           verbose=0, mode='auto', baseline=None,
                           restore_best_weights=True)

'''
This callback allows us to reduce the learning rate when our validation loss
hits a plateau.
'''
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=0)

classifier = BERTForClassification(model, num_classes=1082)

classifier.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=0.0001, weight_decay=0.01),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
# Training the model with callbacks and validation.
history = classifier.fit(
    train_dataset,
    epochs=20,
    callbacks=[early_stop, reduce_lr],
    validation_data=validation_dataset
)

Epoch 1/20
73/73 [==============================] - 132s 1s/step - loss: 3.9767 - accuracy: 0.4568 - val_loss: 1.6061 - val_accuracy: 0.8020 - lr: 1.0000e-04
Epoch 2/20
73/73 [==============================] - 66s 899ms/step - loss: 1.3318 - accuracy: 0.8212 - val_loss: 1.2153 - val_accuracy: 0.8266 - lr: 1.0000e-04
Epoch 3/20
73/73 [==============================] - 65s 896ms/step - loss: 1.0677 - accuracy: 0.8450 - val_loss: 1.2272 - val_accuracy: 0.8365 - lr: 1.0000e-04
Epoch 4/20
73/73 [==============================] - 66s 901ms/step - loss: 0.9532 - accuracy: 0.8523 - val_loss: 1.2091 - val_accuracy: 0.8404 - lr: 1.0000e-04
Epoch 5/20
73/73 [==============================] - 65s 894ms/step - loss: 0.8522 - accuracy: 0.8627 - val_loss: 1.2233 - val_accuracy: 0.8424 - lr: 1.0000e-04
Epoch 6/20
73/73 [==============================] - 65s 893ms/step - loss: 0.7583 - accuracy: 0.8876 - val_loss: 1.2206 - val_accuracy: 0.8453 - lr: 1.0000e-04
Epoch 7/20
73/73 [========================

In [ ]:
# Saving the weights of the trained model to Google Drive.
from google.colab import drive
drive.mount('/content/drive')

classifier.save_weights('/content/drive/My Drive/disease_classifier_model5', save_format='tf')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Evaluating the out-of-sample performance of the model.
classifier.evaluate(test_dataset)

23/23 [==============================] - 6s 275ms/step - loss: 1.3039 - accuracy: 0.8318


[1.3038804531097412, 0.8317955732345581]